# INSTALL PACKAGE

In [35]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\Admin\Desktop\cvworkshop_try


In [ ]:
%conda info

In [36]:
%pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.27 🚀 Python-3.12.2 torch-2.2.1+cpu CPU (12th Gen Intel Core(TM) i9-12900KS)
Setup complete ✅ (24 CPUs, 63.8 GB RAM, 664.4/894.0 GB disk)


In [37]:
from ultralytics import YOLO

from IPython.display import display, Image

_______________________________________________________________________________________________

# LOAD DATASET

In [38]:
datasets_dir = os.path.join(HOME, "datasets")
os.makedirs(datasets_dir, exist_ok=True)
os.chdir(datasets_dir)

%pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="f5pOg5frjQ3XAImBbLxX")
project = rf.workspace("ravensburgweingarten").project("dog_cat-test")
dataset = project.version(1).download("yolov8")

Note: you may need to restart the kernel to use updated packages.
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.27, to fix: `pip install ultralytics==8.0.196`


_______________________________________________________________________________________________

# TRAIN MODEL

In [39]:
%cd {HOME}
data= f"{dataset.location}/data.yaml"
print(data)

c:\Users\Admin\Desktop\cvworkshop_try
c:\Users\Admin\Desktop\cvworkshop_try\datasets\Dog_Cat-Test-1/data.yaml


In [41]:
#%cd {HOME}
model = YOLO("yolov8n.yaml")
print(1)
print(f"{dataset.location}")
results = model.train(data= os.path.join(dataset.location, 'data.yaml'), epochs=3)

#!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=1 imgsz=640 plots=True

1
c:\Users\Admin\Desktop\cvworkshop_try\datasets\Dog_Cat-Test-1
Ultralytics YOLOv8.1.27 🚀 Python-3.12.2 torch-2.2.1+cpu CPU (12th Gen Intel Core(TM) i9-12900KS)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=c:\Users\Admin\Desktop\cvworkshop_try\datasets\Dog_Cat-Test-1\data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

train: Scanning C:\Users\Admin\Desktop\cvworkshop_try\datasets\Dog_Cat-Test-1\train\labels.cache... 321 images, 0 backgrounds, 0 corrupt: 100%|██████████| 321/321 [00:00<?, ?it/s]
val: Scanning C:\Users\Admin\Desktop\cvworkshop_try\datasets\Dog_Cat-Test-1\valid\labels.cache... 43 images, 0 backgrounds, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]

Plotting labels to runs\detect\train9\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train9
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3         0G      3.136      3.496      4.193          4        640: 100%|██████████| 21/21 [00:50<00:00,  2.41s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.31s/it]

...


        3/3         0G      3.065      3.374      4.014          3        640: 100%|██████████| 21/21 [01:03<00:00,  3.01s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.34s/it]

                   all         43         50    0.00341       0.88      0.196     0.0502

3 epochs completed in 0.047 hours.
Optimizer stripped from runs\detect\train9\weights\last.pt, 6.2MB


Optimizer stripped from runs\detect\train9\weights\best.pt, 6.2MB

Validating runs\detect\train9\weights\best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.12.2 torch-2.2.1+cpu CPU (12th Gen Intel Core(TM) i9-12900KS)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.09s/it]


                   all         43         50    0.00349        0.9        0.3     0.0582
Speed: 1.0ms preprocess, 43.4ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs\detect\train9


_______________________________________________________________________________________________

# EVALUATE MODEL

In [42]:
%cd {HOME}
results = model.val()

c:\Users\Admin\Desktop\cvworkshop_try
Ultralytics YOLOv8.1.27 🚀 Python-3.12.2 torch-2.2.1+cpu CPU (12th Gen Intel Core(TM) i9-12900KS)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\Admin\Desktop\cvworkshop_try\datasets\Dog_Cat-Test-1\valid\labels.cache... 43 images, 0 backgrounds, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.42it/s]


                   all         43         50    0.00349        0.9        0.3     0.0582
Speed: 2.1ms preprocess, 39.4ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\train92


In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

_______________________________________________________________________________________________

# TEST MODEL

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt source={dataset.location}/test/images save=True